In [1]:
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
iris.target_names # Выбираем только последние два

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [3]:
X=iris.data[50:]
y=iris.target[50:]-1 #Меняем нумерацию классов с (1,2) на (0,1)

In [40]:
#Градиентный спуск
t0=1
t1=1
t2=1
t3=1
t4=1 # Начальные значения параметров
step=0.1 # Величина шага
for k in range(2000):
        der_loss_array=np.array([
            np.array([X[i][j]*(y[i]-1/(1+np.exp(-t0-t1*X[i][0]-t2*X[i][1]-t3*X[i][2]-t4*X[i][3]))) 
                      for j in range(len(X[0]))]) for i in range(len(y))]) # Матрица производных
        t1=t1+step*np.array([der_loss_array[i][0] for i in range(len(y))]).sum()
        t2=t2+step*np.array([der_loss_array[i][1] for i in range(len(y))]).sum()
        t3=t3+step*np.array([der_loss_array[i][2] for i in range(len(y))]).sum()
        t4=t4+step*np.array([der_loss_array[i][3] for i in range(len(y))]).sum()
        t0=t0+step*np.array([y[i]-1/(1+np.exp(-t0-t1*X[i][0]-t2*X[i][1]-t3*X[i][2]-t4*X[i][3])) 
                              for i in range(len(y))]).sum()

y_pred=np.array([1/(1+np.exp(-t0-t1*X[i][0]-t2*X[i][1]-t3*X[i][2]-t4*X[i][3])) for i in range(len(y))])
error=((y-y_pred)**2).mean()
print('Среднеквадратичная ошибка:',error)

Среднеквадратичная ошибка: 0.018422678415863807


In [41]:
print(t0,t1,t2,t3,t4)

-66.0300313783441 -82.28828342103637 -79.22486199778989 124.1514248142746 113.8220789039461


In [56]:
# Nesterov momentum
t0=1
t1=1
t2=1
t3=1
t4=1 # Начальные значения параметров
step=0.01 # Величина шага
gamma=0.975 # Коэффициент сохранения
v0=np.array([0.1,0.1,0.1,0.1,0.1])
v1=np.array([0.1,0.1,0.1,0.1,0.1])
for k in range(1000):
    v0=v1
    der_loss_array=np.array([
            np.array([X[i][j]*(y[i]-1/(1+np.exp(-(t0-gamma*v0[0])-(t1-gamma*v0[1])*X[i][0]-(t2-gamma*v0[2])*X[i][1]
                                                -(t3-gamma*v0[3])*X[i][2]-(t4-gamma*v0[4])*X[i][3]))) 
                      for j in range(len(X[0]))]) for i in range(len(y))]) # Матрица производных
    
    v1[0]=gamma*v0[0]+step*(1-gamma)*(np.array([(y[i]-1/(1+np.exp(-(t0-gamma*v0[0])-(t1-gamma*v0[1])*X[i][0]-
                                                                 (t2-gamma*v0[2])*X[i][1]-(t3-gamma*v0[3])*X[i][3]
                                                                 -(t4-gamma*v0[4])*X[i][3]))) for i in range(len(y))]).sum())
    for j in range(1,5):
        v1[j]=gamma*v0[j]+step*(1-gamma)*np.array([der_loss_array[i][j-1] for i in range(len(y))]).sum()
    t1=t1+v1[1]
    t2=t2+v1[2]
    t3=t3+v1[3]
    t4=t4+v1[4]
    t0=t0+v1[0]

y_pred=np.array([1/(1+np.exp(-t0-t1*X[i][0]-t2*X[i][1]-t3*X[i][2]-t4*X[i][3])) for i in range(len(y))])
error=((y-y_pred)**2).mean()
print('Среднеквадратичная ошибка:',error)

Среднеквадратичная ошибка: 0.3796373302716973


In [59]:
# RMSProp
t0=1
t1=1
t2=1
t3=1
t4=1 # Начальные значения параметров
step=0.1 # Величина шага
gamma=0.9 # Коэффициент сохранения
e=1 # Сглаживающий параметр
Egt=np.array([0,0,0,0,0]) # Сумма квадратов смещений
for k in range(500):
        der_loss_array=np.array([
            np.array([X[i][j]*(y[i]-1/(1+np.exp(-t0-t1*X[i][0]-t2*X[i][1]-t3*X[i][2]-t4*X[i][3]))) 
                      for j in range(len(X[0]))]) for i in range(len(y))]) # Матрица производных
        
        for j in range(1,5):
            Egt[j]=Egt[j]*gamma+(1-gamma)*np.array([der_loss_array[i][j-1]**2 for i in range(len(y))]).sum()
        Egt[0]=Egt[0]*gamma+(1-gamma)*np.array([(y[i]-1/(1+np.exp(-t0-t1*X[i][0]-t2*X[i][1]-t3*X[i][2]-t4*X[i][3])))**2
                                               for i in range(len(y))]).sum()
        
        t1=t1+step*np.array([der_loss_array[i][0] for i in range(len(y))]).sum()/np.sqrt(Egt[1]+e)
        t2=t2+step*np.array([der_loss_array[i][1] for i in range(len(y))]).sum()/np.sqrt(Egt[2]+e)
        t3=t3+step*np.array([der_loss_array[i][2] for i in range(len(y))]).sum()/np.sqrt(Egt[3]+e)
        t4=t4+step*np.array([der_loss_array[i][3] for i in range(len(y))]).sum()/np.sqrt(Egt[4]+e)
        t0=t0+step*np.array([y[i]-1/(1+np.exp(-t0-t1*X[i][0]-t2*X[i][1]-t3*X[i][2]-t4*X[i][3])) 
                              for i in range(len(y))]).sum()/np.sqrt(Egt[0]+e)

y_pred=np.array([1/(1+np.exp(-t0-t1*X[i][0]-t2*X[i][1]-t3*X[i][2]-t4*X[i][3])) for i in range(len(y))])
error=((y-y_pred)**2).mean()
print('Среднеквадратичная ошибка:',error)

Среднеквадратичная ошибка: 0.04050109376736006
